Automated Project: Planning, Estimation, and Allocation

## Initial Imports

In [50]:
# Warning control
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import yaml
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
import os




## Set OpenAI Model

In [51]:
# Load environment variables

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

## Loading Tasks and Agents YAML files

In [52]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [53]:
from typing import List
from pydantic import BaseModel, Field

class Answer(BaseModel):
    department: str = Field(..., description="Department of the internal message")
    internal_message: str = Field(..., description="Description of the internal message")
    is_sufficient: bool = Field(..., description="Indicates if the answer is sufficient")
    answer: str = Field(..., description="The answer to the internal message")

class ResponseAnswer(BaseModel):
    response: List[Answer] = Field(..., description="List of answers to internal messages")


In [54]:
# Creating Agents
customer_agent = Agent(
  config=agents_config['customer_agent']
)

retriever_agent = Agent(
  config=agents_config['retriever_agent']
)

responder_agent = Agent(
  config=agents_config['responder_agent']
)

# Creating Tasks
intake_task = Task(
  config=tasks_config['intake_task'],
  agent=customer_agent
)

retrieval_task = Task(
  config=tasks_config['retrieval_task'],
  agent=retriever_agent
)

response_task = Task(
  config=tasks_config['response_task'],
  agent=responder_agent,
  output_pydantic=ResponseAnswer # This is the structured output we want
)

# Creating Crew
crew = Crew(
  agents=[
    customer_agent,
    retriever_agent,
    responder_agent
  ],
  tasks=[
    intake_task,
    retrieval_task,
    response_task
  ],
  verbose=True,
  memory=True,
  max_rpm=10
)

## Crew's Inputs

In [55]:
customer_message = "I was charged twice for my last purchase and I need a refund."


In [56]:

inputs = {
  'customer_message': customer_message
}


## Kicking off the crew

In [57]:

# Run the crew
result = crew.kickoff(
  inputs = inputs
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c480effe-e825-4821-a851-b823bfddca3f                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Intake Agent                                                                                   │
│                                                                                                                 │
│  Task: Analyze the following customer inquiry and determine its intent and relevant department:                 │
│  "{I was charged twice for my last purchase and I need a refund.}"                                              │
│  Create a short internal summary and suggest which department should handle it.                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Knowledge Finder                                                                                        │
│                                                                                                                 │
│  Task: Identify main issue, customer intent, and suggest relevant department                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Responder                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "response": [                                                                                                │
│      {                                                                                                          │
│        "department": "Finance Department",                                                                      │
│        "internal_message": "The Finance Department will handle the issue of the duplicate charge and refund     │
│  processing.",                                                                                                  │
│        "is_sufficient": true,                                                                                   │
│        "answer": "Dear Customer, Thank you for bringing this to our attention. To address the duplicate charge  │
│  on your last purchase, our Finance Department will review the transaction records, verify the duplicate        │
│  charge, and initiate the refund process promptly. We kindly request you to provide any necessary               │
│  documentation, such as proof of payment, to expedite the refund process. Rest assured, we will keep you        │
│  informed about the timeline for the refund and work towards resolving this matter to your satisfaction. Your   │
│  understanding and cooperation are greatly appreciated."                                                        │
│      }                                                                                                          │
│    ]                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 03399462-f542-4e7a-bb66-6882bd583078                                                                     │
│  Agent: Customer Responder                                                                                      │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c480effe-e825-4821-a851-b823bfddca3f                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "response": [                                                                                                │
│      {                                                                                                          │
│        "department": "Finance Department",                                                                      │
│        "internal_message": "The Finance Department will handle the issue of the duplicate charge and refund     │
│  processing.",                                                                                                  │
│        "is_sufficient": true,                                                                                   │
│        "answer": "Dear Customer, Thank you for bringing this to our attention. To address the duplicate charge  │
│  on your last purchase, our Finance Department will review the transaction records, verify the duplicate        │
│  charge, and initiate the refund process promptly. We kindly request you to provide any necessary               │
│  documentation, such as proof of payment, to expedite the refund process. Rest assured, we will keep you        │
│  informed about the timeline for the refund and work towards resolving this matter to your satisfaction. Your   │
│  understanding and cooperation are greatly appreciated."                                                        │
│      }                                                                                                          │
│    ]                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [58]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0004


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,2915,2493,0,422,5


## Result

In [59]:
result.pydantic.dict()

{'response': [{'department': 'Finance Department',
   'internal_message': 'The Finance Department will handle the issue of the duplicate charge and refund processing.',
   'is_sufficient': True,
   'answer': 'Dear Customer, Thank you for bringing this to our attention. To address the duplicate charge on your last purchase, our Finance Department will review the transaction records, verify the duplicate charge, and initiate the refund process promptly. We kindly request you to provide any necessary documentation, such as proof of payment, to expedite the refund process. Rest assured, we will keep you informed about the timeline for the refund and work towards resolving this matter to your satisfaction. Your understanding and cooperation are greatly appreciated.'}]}